# Hypothesis Testing

**Hypothesis Testing Steps:**

1. State null and alternative hypotheses and significance level.
2. Assume that the null hypothesis is true, and choose  a statistic to calculate based on your observed values.
3. Determine/estimate how your chosen statistic is distributed under the null hypothesis
4. Find the $p$-value: how often would you see a sample statistic as extreme or more extreme than the one you observed?
5. If $p$-value is smaller than the significance level, reject the null hypothesis. Otherwise, do not reject the null hypothesis.

## Hypothesis Testing for a Single Mean

The American Time Use Survey (ATUS) measures the amount of time people spend doing various activities, such as paid work, childcare, volunteering, and socializing.

Based on this survey, Americans reported that on average, they spend 527 minutes sleeping.

You suspect that unemployed people will spend more time than the average person sleeping. To try and determine if this is true, you have a sample of the amount of time spent sleeping as reported by 20 unemployed participants contained in the `time_use_sample.csv` file.

**Null and alternative hypotheses:**

Null hypothesis: unemployed people sleep the same amount on average as the general population.

$H_0: \mu = 527$

Alternative hypothesis, unemployed people sleep more on average than the general population.

$H_1: \mu > 527$

This is a **one-tailed** test, since you are only interested in testing is the average is greater than 527, rather than if it is not equal to 527.

Also, you need to specify our significance level. Here, you can use the standard significance level of 0.05.

Now, let's look at the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
time_use_sample = pd.read_csv('../data/time_use_sample.csv')

In [ ]:
time_use_sample.head()

Let's take a look at the distribution of reported sleeping times.

In [ ]:
time_use_sample.minutes_spent_sleeping.hist();

In [ ]:
time_use_sample['minutes_spent_sleeping'].describe()

On average, in this sample, people reported sleeping for an average of 582.5 minutes, but there was quite a bit of variability, with the standard deviation being 176 minutes.

To run the hypothesis test, you need to quantify how unusual the observed values would be if the null hypothesis were true. You can do this by looking at what the **sampling distribution of the mean** would be, under the null hypothesis. Recall from the confidence intervals section that, as long as the population is approximately normal, or you have a large enough sample size, you can use a $t$-distribution to understand the sampling distribution of the mean.

Specifically, 
$$t = \frac{\bar{x} - \mu}{s/\sqrt{n}}$$ 

follows a $t$-distribution with $n-1$ degrees of freedom, where $n$ is the sample size. Here $t$ is your **test statistic** and will be the basis for determining whether or not to reject the null hypothesis.

The sample distribution in this case is slightly skewed to the right, but looks close enough to normal that you should be okay in using a $t$-test.

In [ ]:
from scipy.stats import t

Now, calculate the test statistic so you can see where it lands on this plot.

In [ ]:
x_bar = time_use_sample.minutes_spent_sleeping.mean()
s = time_use_sample.minutes_spent_sleeping.std()

t_value = (x_bar - 527)/(s / np.sqrt(len(time_use_sample)))
print(t_value)

What you see is that the observed value is 1.407. Just how unusual would this be, if the null hypothesis were true? First, let's see where the test statistic lands on the sampling distribution of the mean. You can do this using the hypot_plot_mean function imported from nssstats.plots.

In [ ]:
from nssstats.plots import hypot_plot_mean 

In [ ]:
hypot_plot_mean(time_use_sample.minutes_spent_sleeping, popmean=527, type = 'right', area = False)

Recall that the $p$-value quantifies how unusual an observation would be if the null hypothesis were true. That is, how likely it would be to see a sample mean as extreme or more extreme than the observation. Since you are doing a one-tailed test, more extreme would correspond to a sample test statistic to the right of our observed test statistic. How likely is that? The likelihood is equal to the area under the curve to the right of the observed test statistic.

You can find that area using the cumulative distribution function for the $t$ distribution with 19 degrees of freedom (one fewer than the sample size). Since you are calculating area to the right, you need to use the trick where you subtract the cdf value from 1.

In [ ]:
1 - t.cdf(t_value, df = len(time_use_sample) - 1)

In [ ]:
hypot_plot_mean(time_use_sample.minutes_spent_sleeping, popmean=527, type = 'right')

What you can see is that, if the null hypothesis is true, you would see something at least as extreme as our sample mean almost 9% of the time. This is not less than the threshold of 5%, so you should not reject the null hypothesis. You do not have sufficient evidence to conslude that unemployed people sleep more on average than the general population. 

Above, you manually calculated the test statistic and $p$-value, but in practice you can use the `ttest_1samp` function from scipy stats.

In [ ]:
from scipy.stats import ttest_1samp

To use this function, you need to pass in the column containing the data you want to test. In this case, that is `time_use_sample.minutes_spent_sleeping`. You also need to pass in the population mean under the null hypothesis. In this case, the hypothesized mean is 527.

Warning - the function `ttest_1samp` does only two-sided tests. This means that you are checking whether the mean is not equal to the proposed population mean. By symmetry of $t$-distributions, you can adjust by cutting the reported $p$-value in half.

In [ ]:
t_result = ttest_1samp(time_use_sample.minutes_spent_sleeping, popmean = 527)
t_result

What does this mean? Well, first you must divide the reported $p$-value in half since you are conducting a one-sided test.

In [ ]:
t_result.pvalue / 2

Based on the results of this test, you see that, even if the true mean is only 527, you will see a sample mean of at least 582.5 8.77% of the time. 

The standard is to only reject the null hypothesis if there is less than a 5% chance of the sample mean being as large as it was. In this case, you do not have enough evidence to reject the null hypothesis and conclude that unemployed people get more sleep than the general population.

Let's try to understand why, even though there was such large difference (almost an hour), you were not able to conclude that this difference really exists. It has to do with two factors:

* First, you had a very large sample standard deviation (120 minutes). When there is a lot of variability in the population, you get a wider sampling distribution, meaning that is it more likely for a given sample mean to be much different from the population mean.

* Second, you have a small sample size. The sampling distribution of the mean's variance is inversely proportional to the sample size, so a small sample size results in a large variance in the sampling distribution, and hence, more likelihood of sample means far from the population mean.

Let's repeat this analysis, but this time with a larger sample size.

In [ ]:
time_use_sample_larger = pd.read_csv('../data/time_use_sample_b.csv')

In [ ]:
time_use_sample_larger.minutes_spent_sleeping.describe()

In [ ]:
time_use_sample_larger.minutes_spent_sleeping.hist();

**Using same null and alternative hypotheses as above, use `ttest_1samp` to find the $p$-value and state your conclusion.**

In [ ]:
# Your code here

**State your conclusion here.**

## Hypothesis Testing for Difference in Means

Oftentimes, you will be testing to see if there is a difference between two populations. For example, you might want to compare amount of time spent sleeping by the male population vs the female population from the American Time Use Survey. In this case, you won't specify a direction for the difference. That is, you won't test that males get more sleep than females or vice versa. You'll only test that there is some difference. This means that you are doing a two-tailed test.

Before bringing in the data, let's state the null and alternative hypotheses. Remember that the null hypothesis says that there is no difference between the population means. Let $\mu_M$ represent the average time spent sleeping by males and $\mu_F$ represent the average time spent sleeping by females.

**Null Hypothesis:**

$H_0: \mu_M = \mu_F$

**Alternative Hypothesis:**

$H_1: \mu_M \neq \mu_F$

Again, you can use the 0.05 significance level.

Now, you can bring in the data, which is a sample of 25 men and 25 women along with the amount of time they reported sleeping.

In [ ]:
sleeping = pd.read_csv('../data/atus_sleeping.csv')

First, look at some summary statistics.

In [ ]:
sleeping.groupby('sex')['minutes_spent_sleeping'].describe()

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(data = sleeping, x = 'sex', y = 'minutes_spent_sleeping');

You also need to look at the data to ensure that it looks reasonably normally distributed.

In [ ]:
sleeping.loc[sleeping['sex'] == 'Male', 'minutes_spent_sleeping'].hist();

In [ ]:
sleeping.loc[sleeping['sex'] == 'Female', 'minutes_spent_sleeping'].hist();

There does appear to be a difference between males and females in terms of the amount of time spent sleeping. However, there is quite a bit of variability in the dataset, so you need to check to see how likely the difference that you observe is due simply to the randomness inherent in sampling.

The next thing you need is a test statistic and an understanding of how it is distributed. You can use a test statistic derived by Welch.

**Fact:** If both populations are approximately normally distributed, then 

$$t = \frac{\bar{x}_1 - \bar{x}_2}{s}$$ approximately follows a $t$ distribution. Here

$$s = \sqrt{\frac{s_1^2}{N_1} + \frac{s_2^2}{N_2}}$$

where $s_1$ and $s_2$ are the sample standard deviations and $N_1$ and $N_2$ are the sample sizes.

This $t$ distribution has degrees of freedom equal to 
$$df = \left(\frac{s_1^2}{n_1}+ \frac{s_2^2}{n_2}\right)^2 / \left(\frac{s_1^4}{n_1^2(n_1-1)} + \frac{s_2^4}{n_2^2(n_2-1)} \right)$$

See http://www.dcscience.net/welch-1947.pdf if you really want to see details, but I don't recommend it.

Luckily for us, this has been implemented in `scipy.stats` as `ttest_ind`.

In [ ]:
from scipy.stats import ttest_ind

To use this function, you need to pass in the observed values for each group and specify that you want `equal_var = False`.

In [ ]:
t_results = ttest_ind(sleeping.loc[sleeping.sex == 'Male', 'minutes_spent_sleeping'],
                      sleeping.loc[sleeping.sex == 'Female', 'minutes_spent_sleeping'],
                     equal_var = False)
t_results

By default, this will conduct a two-sided test, which, in this case is what you want. Let's see how this looks on the distribution of the test statistic.

In [ ]:
from nssstats.plots import hypot_plot_mean_2sample

In [ ]:
hypot_plot_mean_2sample(sleeping.loc[sleeping.sex == 'Male', 'minutes_spent_sleeping'],
                       sleeping.loc[sleeping.sex == 'Female', 'minutes_spent_sleeping'], type='both')

**State your conclusion at the 0.05 significance level.**

## Hypothesis Testing for a Single Proportion

You can test not only a hypothesis about a mean, but also about a proportion. This can be done using the `proportions_ztest` from the `statsmodels` library. 

In [ ]:
from scipy.stats import binom

Let's start with the coin example from the slides, where you suspected that our coin was not a fair coin. Recall from the slides that there were two scenarios.

**Scenario 1:** You flip the coin 100 times and get the following outcome:

<table width="150" border="3" style="border: 1px solid black"> 
    <tr> 
        <th colspan="2"style="background-color:lightblue"><div align="center">Observed Outcome</div></th> 
    </tr> 
    <tr> 
        <td style="border: 1px solid black">Heads</td> 
        <td style="border: 1px solid black">47</td> 
    </tr> 
    <tr> 
        <td style="border: 1px solid black">Tails</td> 
        <td style="border: 1px solid black">53</td> 
    </tr> 
</table> 

The first thing you need to do is to state our null and alternative hypothesis.

**Null Hypothesis:** The coin is eqully likely to land on heads and tails.
$$H_0: \text{probability of heads} = 0.5$$

**Alternative Hypothesis:** The coin is more likely to land on one side of the other.
$$H_1: \text{probability of heads} \neq 0.5$$


To find the $p$-value, you can use the binomial distribution. Recall that the $p$-value is the probability of getting an as extreme or more extreme outcome, if the null hypothesis were true.

**Question:** Which outcomes (number of heads) would be more be more extreme than the observed outcome?

**Your answer here.**

**Now, use the binomial distribution to calculate the probability of these more extreme outcomes. That is, find the $p$-value.**

In [ ]:
#Your code here

**State your conclusion at the 0.05 significance level.**

Since the $p$-value is larger than 0.05, there is not enough evidence to reject the null hypothesis.

Now, using the same null and alternative hypothesis, repeat this for scenario two:

<table width="150" border="3" style="border: 1px solid black"> 
    <tr> 
        <th colspan="2"style="background-color:lightblue"><div align="center">Observed Outcome</div></th> 
    </tr> 
    <tr> 
        <td style="border: 1px solid black">Heads</td> 
        <td style="border: 1px solid black">38</td> 
    </tr> 
    <tr> 
        <td style="border: 1px solid black">Tails</td> 
        <td style="border: 1px solid black">62</td> 
    </tr> 
</table> 

**What outcomes (number of heads) would be considered as extreme or more extreme that what was observed?**

**Your answer here.**

**Use the binomial distribution to find the $p$-value.**

In [ ]:
# Your code here

**State your conclusion at the 0.05 significance level.**

**Your conclusion here.**

You can also do one-tailed tests for proportions. Let's say you have a die that we think might be loaded and suspect that it is more likely to land on 6. 

**Start by stating the null and alternative hypotheses (if we want to conduct a one-tailed test)**.

$$H_0: \text{probability of rolling a six} \text{  <fill in the rest>}$$

$$H_1: \text{probability of rolling a six} \text{  <fill in the rest>} $$

You roll it 40 times and count 10 times that it lands on 6. 

**Which outcomes would be considered as extreme or more extreme than what we observed?**

**Your answer here**

**Use a binomial distribution to find the $p$-value**.

In [ ]:
#Your code here

**State your conclusion at the 0.05 significance level.**

**Your conclusion here.**

## Hypothesis Testing for the Difference in Proportions

Finally, let's look at the squirrel census data again. A lot of people have never seen a black squirrel, so me might hypothesize that black squirrels are more skittish and run from humans more frequently.

Let's formally state this as a null and alternative hypothesis.

**Null Hypothesis:** There is no difference in the proportion of black squirrels who run compared to the proportion of other other colors of squirrels who run.
$$H_0: p_{black} = p_{other}$$

**Alternative Hypothesis:** The proportion of black squirrels who run is higher than the proportion of other color colors of squirrels who run.
$$H_1: p_{black} > p_{other}$$

Now, let's bring in the data.

In [ ]:
squirrels = pd.read_csv('../data/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv')

In [ ]:
# Remove the rows with missing values for the Fur Color
squirrels = squirrels[~squirrels['Primary Fur Color'].isna()]

# Create a Boolean column indicating whether a squirrel's primary fur color is black
squirrels['Black'] = squirrels['Primary Fur Color'] == 'Black'

Run a cross-tabulation to see how often black squirrels run compared to other colors.

In [ ]:
ct = pd.crosstab(squirrels['Runs from'], squirrels['Black'])
ct

The columns of this table indicate whether a squirrel was black or not and the rows indicate whether that squirrel ran.

Now, normalize by column to see the results as proportions.

In [ ]:
pd.crosstab(squirrels['Runs from'], squirrels['Black'], normalize='columns')

You can see that black squirrels were observed to be more likely to run. But is there enough evidence to conclude that this holds for the entire squirrel population?

To calculate our $p$-value, you can use the `fisher_exact` from `scipy.stats`.

In [ ]:
from scipy.stats import fisher_exact

To use `fisher_exact`, you just need to pass in the non-normalized cross-tabulation and specify the alternative hypothesis ('less', 'greater', or 'two-sided').

To determine the correct alternative hypothesis, just ask "What relationship am I testing between the bottom right entry and bottom left entry of my cross-tabulation?"

In this case, the bottom right corresponds to the proportion of black squirrels that ran away and the bottom left correspond to the proportion of other squirrels that ran away. Since my alternative hypotheis is that black squirrels are more likely to run, I will use `alternative = 'greater'`.

In [ ]:
fisher_exact(ct, alternative = 'greater')

The second value returned is the $p$-value.

**State your conclusion at the 0.05 significance level.**

